In [ ]:
# check GPU
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

In [ ]:
# setup environment
!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.8 2
!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.9 1
!sudo apt install python3.8

!sudo apt-get install python3.8-distutils

!python --version

!apt-get update

!apt install software-properties-common

!sudo dpkg --remove --force-remove-reinstreq python3-pip python3-setuptools python3-wheel

!apt-get install python3-pip

print('Git clone project and install requirements...')
!git clone https://github.com/Winfredy/SadTalker &> /dev/null
%cd SadTalker
!export PYTHONPATH=/content/SadTalker:$PYTHONPATH
!python3.8 -m pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
!apt update
!apt install ffmpeg &> /dev/null
!python3.8 -m pip install -r requirements.txt

In [ ]:
print('Download pre-trained models...')
!rm -rf checkpoints
!bash scripts/download_models.sh

In [ ]:
# Extract text from webpage
import requests
from bs4 import BeautifulSoup

url = 'https://azluminaria.org/2025/05/02/longer-paid-parking-hours-coming-to-most-downtown-tucson-districts-this-summer/'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    paragraphs = soup.find_all('p')
    text = ' '.join([p.get_text() for p in paragraphs])
    text = text.strip().replace('\n', ' ')[:500]
    print("Extracted text:", text)
else:
    print(f'Failed to retrieve the page. Status code: {response.status_code}')


In [ ]:
# Convert Text to Speech
import pyttsx3
import os

# Initialize TTS engine
engine = pyttsx3.init()
engine.setProperty('rate', 150)
engine.setProperty('volume', 1.0)

text = "This is a sample news paragraph to be converted into speech."

# Define output path in SadTalker directory
output_path = 'SadTalker/examples/driven_audio/news_audio.wav'

# Save audio
engine.save_to_file(text, output_path)
engine.runAndWait()

print(f"Audio saved to: {output_path}")


In [ ]:
from shutil import copyfile
import os

!mkdir -p examples/source_image examples/driven_audio

# upload image
source_img_path = "/content/your_image.png"
copyfile(source_img_path, "examples/source_image/myface.png")
copyfile("news_audio.wav", "examples/driven_audio/voice.wav")


In [ ]:
# selected audio from exmaple/driven_audio
img = 'examples/source_image/{}.png'.format(default_head_name.value)
print(img)
!python3.8 inference.py --driven_audio ./examples/driven_audio/news_audio.wav \
           --source_image {img} \
           --result_dir ./results --still --preprocess full --enhancer gfpgan

In [ ]:
# visualize code from makeittalk
from IPython.display import HTML
from base64 import b64encode
import os, sys

# get the last from results

results = sorted(os.listdir('./results/'))

mp4_name = glob.glob('./results/*.mp4')[0]

mp4 = open('{}'.format(mp4_name),'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

print('Display animation: {}'.format(mp4_name), file=sys.stderr)
display(HTML("""
  <video width=256 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url))